In [ ]:
# Import necessari
import cv2
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import json
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load the JSON data
with open('Dataset_combined/588786395.json', 'r') as f:
    data = json.load(f)

# Extract the shapes from the JSON data
shapes = data['shapes']

# Create a figure and axis
fig, ax = plt.subplots()

# Set the aspect ratio to maintain the original aspect ratio of the image
ax.set_aspect('equal')

# Invert y-axis
ax.invert_yaxis()

# Loop through the shapes and plot them
for shape in shapes:
    label = shape['label']
    points = shape['points']
    
    # Create a polygon patch for the shape
    polygon = patches.Polygon(points, closed=True, fill=False, linewidth=2, edgecolor='r')
    ax.add_patch(polygon)

# Set the axis limits to fit the shapes
ax.relim()
ax.autoscale_view()

x_coords = [point[0] for shape in shapes for point in shape['points']]
y_coords = [point[1] for shape in shapes for point in shape['points']]
mask_width = max(x_coords) - min(x_coords)
mask_height = max(y_coords) - min(y_coords)
print('mask width', mask_height)
print('mask height', mask_width)

# Show the plot
plt.show()

import matplotlib.image as mpimg

# Load and plot an image
img = mpimg.imread('Dataset/588786395_training.png')  # Sostituisci 'path_to_your_image.png' con il percorso dell'immagine PNG
plt.imshow(img)
plt.axis('off')  # Nasconde gli assi
height, width, _ = img.shape

print("Dimensioni dell'immagine:")
print("Altezza:", height)
print("Larghezza:", width)
plt.show()

from PIL import Image

# Carica l'immagine
img = Image.open('Dataset/588786395_training.png')  # Sostituisci 'path_to_your_image.png' con il percorso dell'immagine PNG

# Scalare l'immagine alle nuove dimensioni
new_width = 440
new_height = height+(mask_width-width)
print(new_width)
print(new_height)
resized_img = img.resize((int(new_width), int(new_height)))

plt.imshow(resized_img)
plt.axis('off')
plt.show()


In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Load the JSON data
with open('Dataset_combined/588786395.json', 'r') as f:
    data = json.load(f)

# Extract the shapes from the JSON data
shapes = data['shapes']

# Create a figure and axis
fig, ax = plt.subplots()

# Set the aspect ratio to maintain the original aspect ratio of the image
ax.set_aspect('equal')

# Invert y-axis
ax.invert_yaxis()

# Loop through the shapes and plot them
for shape in shapes:
    label = shape['label']
    points = shape['points']
    
    # Create a polygon patch for the shape
    polygon = patches.Polygon(points, closed=True, fill=False, linewidth=2, edgecolor='r')
    ax.add_patch(polygon)

# Set the axis limits to fit the shapes
ax.relim()
ax.autoscale_view()

from PIL import Image

# Carica l'immagine originale
img = Image.open('Dataset/588786395_training.png')  # Sostituisci 'path_to_your_image.png' con il percorso dell'immagine originale

# Altezza desiderata per la nuova immagine
new_height = 1024

# Calcola la nuova larghezza mantenendo il rapporto di aspetto
width, height = img.size
aspect_ratio = width / height
new_width = int(aspect_ratio * new_height)
print(new_height, new_width)

# Ridimensiona l'immagine
resized_img = img.resize((new_width, new_height))

# Salva o mostra l'immagine ridimensionata
#resized_img.show()
#resized_img.save('path_to_save_resized_image.png')  # Sostituisci 'path_to_save_resized_image.png' con il percorso in cui desideri salvare l'immagine ridimensionata


# Load the original image
#original_image = Image.open('Dataset/387805419_training.png')  # Sostituisci 'path_to_your_original_image.png' con il percorso dell'immagine originale
# Resize the original image
#new_width = 440
#new_height = 468
#resized_image = original_image.resize((new_width, new_height))

# Show the resized image
plt.imshow(resized_img)

# Show the plot
plt.axis('off')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Carica le immagini
img1 = mpimg.imread('Dataset/588786395_training.png')
img2 = mpimg.imread(r'C:\Users\loren\Downloads\APKLOT-master\APKLOT-master\1. Satellite\Dataset\parcheggi\segmented_images\588786395_SegmentationClass.png')

# Imposta il grado di trasparenza per ciascuna immagine
alpha = 1.0  # Modifica questo valore per controllare il grado di trasparenza

# Sovrapponi le due immagini
blended_img = alpha * img1 + (1 - alpha) * img2

# Plotta l'immagine blendata
plt.imshow(blended_img)
plt.axis('off')  # Nascondi gli assi
plt.show()


In [ ]:
import os
import glob
from PIL import Image
import os
import glob
from PIL import Image

class ParkingLotDataset(Dataset):
    def __init__(self, root_img, root_msk, pairs=None, transforms=None):
        self.root_img = root_img
        self.root_msk = root_msk
        self.transforms = transforms

        if pairs is None:
            # Get all image files
            self.image_paths = sorted(glob.glob(os.path.join(root_img, '*.png')))

            # Get all mask files
            self.mask_paths = sorted(glob.glob(os.path.join(root_msk, '*.png')))

            # Pair image and mask files based on their filenames
            #self.pairs = [(image_path, mask_path) for image_path in self.image_paths for mask_path in self.mask_paths if os.path.splitext(os.path.basename(image_path))[0] == os.path.splitext(os.path.basename(mask_path))[0]]
            self.pairs = []

            for image_path in self.image_paths:
                image_filename = os.path.splitext(os.path.basename(image_path))[0]
                for mask_path in self.mask_paths:
                    mask_filename = os.path.splitext(os.path.basename(mask_path))[0]
                    if image_filename in mask_filename:
                        self.pairs.append((image_path, mask_path))
                        break  # Se trova una corrispondenza, passa al prossimo file immagine
        else:
            self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        image_path, mask_path = self.pairs[idx]

        # Load image
        image = Image.open(image_path)
        image_array = np.array(image)

        # Apply transformations
        if self.transforms:
            image_array = self.transforms(image_array)
            print(image_array.shape)
        

    


        return image_array, masks
        

        #merged_mask = np.zeros((2108, 2113), dtype=np.uint8)

        # Load and plot the masks
        #with open(mask_path, 'r') as f:
        #    masks_data = json.load(f)

        #for shape in masks_data['shapes']:
        #    points = np.array(shape['points'])
        #    mask = np.zeros((2108, 2113), dtype=np.uint8)
        #    cv2.fillPoly(mask, [points.astype(np.int32)], 1)
        #    _, binary_mask = cv2.threshold(mask, 0, 1, cv2.THRESH_BINARY)
        #    merged_mask += binary_mask

        #if self.transforms:
        #    masks = self.transforms(merged_mask)
        #    print(masks.shape)

        #print("Image shape:", image_array.shape)
        #print("Masks shape:", masks.shape)